In [ ]:
%load_ext autoreload
%autoreload 2

import json

import geopandas as gpd
import folium
import folium.plugins
import logging
from json import dumps
from eo_tools.util import explore_products

from eodag import EODataAccessGateway
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

In [ ]:
# explore product types
# [it["ID"] for it in dag.list_product_types()]

In [ ]:
# load a geometry
file_aoi = "/data/aoi_Maroc.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S2_MSI_L1C",
    "start": "2022-04-01",
    "end": "2022-05-30",
    "geom": shp
}
results = dag.search_all(**search_criteria)

In [ ]:
def get_unique_dates(results):
    import numpy as np

    dates = []
    for p in results:
        dates.append(p.properties["startTimeFromAscendingNode"])
    return np.unique(dates).tolist()

dates = get_unique_dates(results)
print(f"unique dates:")
print(", \n".join(dates))

In [ ]:
# show all results
# explore_products(results, shp)

# filter by date
results_20230401 = results.filter_date("2022-04-01", "2022-04-02") 
# explore_products(results_20230401, shp)

In [ ]:
dl = dag.download_all(results_20230401, outputs_prefix="/data/S2/")

In [ ]:
# TODO
# - Load 10m band profile
# - Resample all bands to 10m
# - Convert all to EPSG 4326
# - Crop to AOI
# - Merge products
# - Make GeoTiff

In [ ]:
import rasterio 
import matplotlib.pyplot as plt

ds = rasterio.open(dl[0])
ds0 = rasterio.open(ds.subdatasets[-1])
print(ds.subdatasets)
im0 = ds0.read()
# with rasterio.open(dl[0]) as ds:
    # print(ds.profile)
    # ds.s
    # img = ds.read(1)
plt.figure(figsize=(10,10))
plt.imshow(im0.transpose(1,2,0))
plt.axis('off')

In [ ]:
# import geojson
# import json
# gpd.read_file(json.dumps(results.as_geojson_object()))

### TODO 
- Group products by date
- Download
- Resample all bands to 10m
- Stitch and make geotiffs